<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train5_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:

train5 = pd.read_csv(r'/content/train_data5.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [3]:

train5['Target'] = train5['Target'].replace(['Others'],'Invalid')


In [4]:
train5['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [5]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train5_Xtfidf = tfidf_vect.fit_transform(train5['Sentence'])


In [7]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [8]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [9]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [10]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [11]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [12]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train5_Xtfidf,train5['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6836283185840708
                  precision    recall  f1-score   support

        Analysis       0.53      0.41      0.46       126
      Conclusion       0.78      0.56      0.65        45
           Facts       0.74      0.92      0.82       483
         Invalid       0.52      0.42      0.46       133
           Issue       0.55      0.44      0.49        48
Rule/Law/Holding       0.69      0.26      0.38        69

        accuracy                           0.68       904
       macro avg       0.64      0.50      0.54       904
    weighted avg       0.67      0.68      0.66       904



In [14]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [15]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9919723122394855, 0.9932611579909031, 0.9946162241882684, 0.9999549945637808, 0.9989696945268521, 0.996896046792411, 0.9949886841631062, 0.9991297680946035, 0.9972246337277954, 0.9894562010114011, 0.9931724983814998, 0.9993566734268252, 0.997842641108448, 0.9993145978697282, 0.9887583659126279, 0.9997214574592039, 0.9874028744753104, 0.9823523361321043, 0.9843747322150024, 0.9902902826516475, 0.9919726123260874, 0.9973766082708955, 0.9984113956301057, 0.9995406667229476, 0.9997624982802955, 0.9911748022462012, 0.99666890834534, 0.9920604917088269, 0.9939764438118125, 0.9953320265746016, 0.9959056490933956, 0.9963965722085061, 0.9984915103345059, 0.9872124356453377, 0.9943688351950404, 0.9843298456828432, 0.9998183006568643, 0.9886708871684117, 0.9845991839099518, 0.9904042574508253, 0.9834128098257786, 0.9996311555496274, 0.9958615253929425, 0.99935

In [16]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 32, 33, 35, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 65, 68, 70, 76, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [17]:
frame_1 = [train5,unlabel_1]
train5_1 = pd.concat(frame_1)
len(train5_1)

3521

In [18]:
x_train_1 = tfidf_vect.transform(train5_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train5_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6553191489361702
                  precision    recall  f1-score   support

        Analysis       0.48      0.48      0.48        98
      Conclusion       0.79      0.55      0.65        42
           Facts       0.72      0.86      0.79       344
         Invalid       0.52      0.54      0.53       112
           Issue       0.70      0.48      0.57        48
Rule/Law/Holding       0.61      0.23      0.33        61

        accuracy                           0.66       705
       macro avg       0.64      0.52      0.56       705
    weighted avg       0.65      0.66      0.64       705



In [19]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.996839767999939, 0.9990531014708578, 0.9977748258751561, 0.9932433095441232, 0.9961669907163305, 0.996571353509513, 0.9999376910257127, 0.9921064512619138, 0.999049018011642, 0.9962770273430956, 0.984716820904696, 0.9981736586958615, 0.9955806177205474, 0.998633756557231, 0.9999617362339831, 0.9994050794085524, 0.9935022231333979, 0.9871925856614984, 0.9984658359584405, 0.9867862049682536, 0.99853036759803, 0.9931443638461535, 0.9836023735134248, 0.9995986885915573, 0.987532644219489, 0.9987133239657611, 0.9998968391748589, 0.9987560513648953, 0.9995422796803062, 0.9970496999793959, 0.9938588405340508, 0.9955049271494564, 0.9812723768487697, 0.9985502697670677, 0.9996305643367897, 0.9992934147887549, 0.9811422134510557, 0.9892096492759311, 0.9976098469691363]
[0, 2, 4, 6, 7, 8, 12, 17, 20, 21, 23, 29, 34, 36, 37, 39, 42, 44, 49, 50, 54, 55, 58, 59, 60, 63, 64, 69, 70

In [20]:
unlabel_2 = unlabel_2.loc[[0, 2, 4, 6, 7, 8, 12, 17, 20, 21, 23, 29, 34, 36, 37, 39, 42, 44, 49, 50, 54, 55, 58, 59, 60, 63, 64, 69, 70, 71, 75, 83, 84, 85, 88, 91, 92, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [21]:
frame_2 = [train5_1,unlabel_2]
train5_2 = pd.concat(frame_2)
len(train5_2)

3560

In [22]:
x_train_2 = tfidf_vect.transform(train5_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train5_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6611226611226612
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47       142
      Conclusion       0.69      0.36      0.47        50
           Facts       0.73      0.90      0.81       486
         Invalid       0.54      0.46      0.49       164
           Issue       0.56      0.33      0.42        60
Rule/Law/Holding       0.66      0.35      0.46        60

        accuracy                           0.66       962
       macro avg       0.61      0.48      0.52       962
    weighted avg       0.65      0.66      0.64       962



In [23]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9967365481604431, 0.9982853096827699, 0.9950301314536985, 0.9865942827971659, 0.9996607387799333, 0.9997060496120983, 0.9994144222968923, 0.9996856885738759, 0.9911779136196938, 0.9998611891959067, 0.9980269895897864, 0.998315445279868, 0.9992764321951658, 0.9922379701206105, 0.9993221511639986, 0.998391759324987, 0.9925359364554683, 0.9885214777215785, 0.9971417205316148, 0.9937901864008002, 0.9897046230733222, 0.9985829842014063, 0.9996566533426398, 0.9939676326649364, 0.9999038695123545, 0.9951515660982904, 0.9961439785125074, 0.9995417681422222, 0.9811559876232521, 0.9992705751585819, 0.9890787376228625, 0.9990546466397119, 0.9886198004657206, 0.9918745482601351, 0.9995931907539242, 0.984283934664287]
[0, 2, 3, 4, 7, 8, 12, 20, 26, 27, 29, 30, 35, 36, 43, 46, 47, 48, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 73, 75, 78, 79, 89, 96]
36
36
36


In [24]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 20, 26, 27, 29, 30, 35, 36, 43, 46, 47, 48, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 73, 75, 78, 79, 89, 96],:]
frame_3 = [train5_2,unlabel_3]
train5_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train5_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train5_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6722222222222223
                  precision    recall  f1-score   support

        Analysis       0.44      0.45      0.45        95
      Conclusion       0.71      0.51      0.59        47
           Facts       0.77      0.89      0.83       381
         Invalid       0.49      0.40      0.44       112
           Issue       0.67      0.54      0.60        37
Rule/Law/Holding       0.48      0.25      0.33        48

        accuracy                           0.67       720
       macro avg       0.59      0.51      0.54       720
    weighted avg       0.65      0.67      0.66       720



In [25]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2]
[0.9852238527680338, 0.9996538026285456, 0.9988762927105576, 0.9918374348454161, 0.9962390149538201, 0.9997662834597344, 0.9855715705828496, 0.9967227445671398, 0.9843165344457525, 0.9813949905754294, 0.9997898395237103, 0.99955002128031, 0.9984560609388476, 0.9973620646256818, 0.9991630872059905, 0.999690157757179, 0.9986164908313362, 0.9988358267116795, 0.9996882939190748, 0.9983014708391795, 0.9982963037075326, 0.9867747555633631, 0.9993799441718709, 0.9957143613695857, 0.9885099611870225, 0.991908955009273, 0.9970901648192562, 0.9843770981511364, 0.9975153835524155, 0.9934520156458424, 0.9927219342986655, 0.993013130343558, 0.9887901610406242, 0.9990580860784986, 0.9918133700656022, 0.9996381338552703, 0.9906912040367982, 0.9908715546335283, 0.9999471110067046, 0.9989882768131088, 0.9870778312122189, 0.987893275999908, 0.9804010444793307, 0.9873250865

In [26]:
unlabel_4=unlabel_4.loc[[0, 2, 4, 6, 7, 10, 11, 15, 16, 17, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 42, 48, 49, 50, 54, 56, 58, 60, 64, 66, 67, 69, 70, 71, 72, 75, 87, 93, 94, 97, 99 ],:]

In [27]:
frame_4 = [train5_3,unlabel_4]
train5_4 = pd.concat(frame_4)
len(train5_4)
x_train_4 = tfidf_vect.transform(train5_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train5_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6622583926754833
                  precision    recall  f1-score   support

        Analysis       0.50      0.36      0.42       154
      Conclusion       0.61      0.50      0.55        46
           Facts       0.73      0.91      0.81       511
         Invalid       0.55      0.42      0.48       156
           Issue       0.52      0.38      0.44        60
Rule/Law/Holding       0.55      0.38      0.45        56

        accuracy                           0.66       983
       macro avg       0.58      0.49      0.52       983
    weighted avg       0.64      0.66      0.64       983



In [28]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 0, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 5, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9991487887711659, 0.9980703753481535, 0.9827256021637314, 0.9919783047816468, 0.9823888012936439, 0.990081591266654, 0.9918799449437307, 0.9877915504667719, 0.9989404266973434, 0.9992366344668036, 0.9929244782259692, 0.9824259364275957, 0.9921371919457067, 0.9994527433744338, 0.9851239640340136, 0.9878565127936624, 0.9925398161544414, 0.9996691208766697, 0.9980933576899298, 0.9813129201859794, 0.99803716417464, 0.9878800273891899, 0.9973894140015008, 0.9815466812672711, 0.989221892229015, 0.9882358947340734, 0.9915746989116783, 0.9862680699854981, 0.9999125494493231, 0.9891611916255117, 0.9824897856936269, 0.9991209759263442, 0.9996400716341691, 0.9823496355949547, 0.9917844121256341, 0.9811906941178304, 0.982359895352316]
[3, 7, 12, 16, 17, 18, 19, 22, 23, 26, 27, 29, 32, 36, 40, 46, 52, 53, 60, 63, 65, 67, 73, 74, 75, 77, 80, 81, 82, 86, 92, 94, 95, 99, 100, 103, 104]
37

In [29]:
unlabel_5 = unlabel_5.loc[[3, 7, 12, 16, 17, 18, 19, 22, 23, 26, 27, 29, 32, 36, 40, 46, 52, 53, 60, 63, 65, 67, 73, 74, 75, 77, 80, 81, 82, 86, 92, 94, 95, 99, 100, 103, 104],:]

In [30]:
frame_5 = [train5_4,unlabel_5]
train5_5 = pd.concat(frame_5)
len(train5_5)
x_train_5 = tfidf_vect.transform(train5_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train5_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6878147029204431
                  precision    recall  f1-score   support

        Analysis       0.53      0.46      0.49       147
      Conclusion       0.67      0.51      0.58        47
           Facts       0.76      0.89      0.82       531
         Invalid       0.53      0.45      0.49       147
           Issue       0.60      0.48      0.54        54
Rule/Law/Holding       0.60      0.37      0.46        67

        accuracy                           0.69       993
       macro avg       0.62      0.53      0.56       993
    weighted avg       0.67      0.69      0.67       993



In [31]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6717850287907869


In [32]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6142661500980078, 0.5619470961267758, 0.5782184538210103, None)
